In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import datetime

Data yang lengkap pada file personel:
1. Personel 2
2. POW 4

Data yang lengkap pada file equipment:
1. aircraft 2
2. helicopter 3
3. tank 4
4. apc 5
5. field artilery 6
6. multiple rocked launcher 7
7. drone 10
8. naval ship 11
9. anti-aircraft warfare 12


In [ ]:
data_equipment = pd.read_csv('/content/drive/MyDrive/UAS Kecerdasan Buatan/russia_losses_equipment.csv')
data_personel = pd.read_csv('/content/drive/MyDrive/UAS Kecerdasan Buatan/russia_losses_personnel.csv')

# ambil data yang kolomnya lengkap saja
data_personel = data_personel.iloc[:, [2,4]]
data_equipment = data_equipment.iloc[:, [2,3,4,5,6,7,10,11,12]]

In [ ]:
# gabungkan data personel dan data equipment
data_gabungan = pd.concat([data_personel, data_equipment], axis=1)

# pisahkan data untuk russia dan ukraina
data_ukraina = data_gabungan.iloc[::2, :]
data_russia = data_gabungan.iloc[1::2, :]

In [ ]:
# Pisahkan data menjadi data input dan data target. Target output adalah data untuk hari keesokan harinya
data_ukraina_input = data_ukraina.iloc[:-1, :]
data_ukraina_target = data_ukraina.iloc[1:, :]
data_russia_input = data_russia.iloc[:-1, :]
data_russia_target = data_russia.iloc[1:, :]

In [ ]:
# Pisahkan menjadi data training dan data testing
train_ukraina_input, test_ukraina_input, train_ukraina_target, test_ukraina_target = train_test_split(data_ukraina_input, data_ukraina_target, train_size=0.8)
train_russia_input, test_russia_input, train_russia_target, test_russia_target = train_test_split(data_russia_input, data_russia_target, train_size=0.8)

In [ ]:
# model ukraina
nn1 = MLPRegressor(hidden_layer_sizes=(20, 20, 20), solver='adam', max_iter=500, verbose=False)
model_ukraina = []
for i in range(11):
  model_ukraina.append(nn1.fit(train_ukraina_input.iloc[:, i].to_numpy().reshape(-1, 1), train_ukraina_target.iloc[:, i]))

In [ ]:
# model russia
nn2 = MLPRegressor(hidden_layer_sizes=(20, 20, 20), solver='adam', max_iter=500, verbose=False)
model_russia = []
for i in range(11):
  model_russia.append(nn2.fit(train_russia_input.iloc[:, i].to_numpy().reshape(-1, 1), train_russia_target.iloc[:, i]))

In [ ]:
# means absolute error ukraina
mae_train_ukraina = []
mae_test_ukraina = []
for i in range(11):
  mae_train_ukraina.append(mean_absolute_error(model_ukraina[i].predict(train_ukraina_input.iloc[:, i].to_numpy().reshape(-1, 1)), train_ukraina_target.iloc[:, i]))
  mae_test_ukraina.append(mean_absolute_error(model_ukraina[i].predict(test_ukraina_input.iloc[:, i].to_numpy().reshape(-1, 1)), test_ukraina_target.iloc[:, i]))

print('Mean Absolute Error data training Ukraina:')
print(mae_train_ukraina)
print('Mean Absolute Error data testing Ukraina:')
print(mae_test_ukraina)

In [ ]:
# means absolute error russia
mae_train_russia = []
mae_test_russia = []
for i in range(11):
  mae_train_russia.append(mean_absolute_error(model_russia[i].predict(train_russia_input.iloc[:, i].to_numpy().reshape(-1, 1)), train_russia_target.iloc[:, i]))
  mae_test_russia.append(mean_absolute_error(model_russia[i].predict(test_russia_input.iloc[:, i].to_numpy().reshape(-1, 1)), test_russia_target.iloc[:, i]))

print('Mean Absolute Error data training Russia:')
print(mae_train_russia)
print('Mean Absolute Error data testing Russia:')
print(mae_test_russia)

In [ ]:
# prediksi data hari ke-terakhir + 1
score_ukraina = np.array([])
for i in range(11):
  score_ukraina = np.append(score_ukraina, model_ukraina[i].predict(data_ukraina_target.iloc[-1, i].reshape(-1, 1)))

score_russia = np.array([])
for i in range(11):
  score_russia = np.append(score_russia, model_russia[i].predict(data_russia_target.iloc[-1, i].reshape(-1, 1)))

print('index,', 'skor ukraina,', 'skor russia')

# cek jika perbedaan skor salah satu pihak > 1 juta
index = 1
while max(score_ukraina.sum(), score_russia.sum()) - min(score_ukraina.sum(), score_russia.sum()) < 1000000:
  print(index, round(score_ukraina.sum()), round(score_russia.sum()))
  index = index + 1
  score_ukraina_temp = np.array([])
  score_russia_temp = np.array([])
  for i in range(11):
    score_ukraina_temp = np.append(score_ukraina_temp, model_ukraina[i].predict(score_ukraina[i].reshape(-1, 1)))
    score_russia_temp = np.append(score_russia_temp, model_russia[i].predict(score_russia[i].reshape(-1, 1)))
  score_ukraina = score_ukraina_temp
  score_russia = score_russia_temp

print(index, round(score_ukraina.sum()), round(score_russia.sum()))

# print nama pemenang
if max(score_ukraina.sum(), score_russia.sum()) == score_ukraina.sum():
  print('Pemenang = Russia')
else:
  print('Pemenang = Ukraina')

# print skor akhir:
print('Skor Akhir')
print('Ukraina: ', round(score_ukraina.sum()))
print('Russia: ', round(score_russia.sum()))

In [ ]:
# ambil tanggal terakhir pada data
tanggal_akhir_data_str = pd.read_csv('/content/drive/MyDrive/UAS Kecerdasan Buatan/russia_losses_equipment.csv').iloc[-1:, 0].to_string(index=False)
# masukkan ke object datetime
tanggal_akhir_data = datetime.datetime.strptime(tanggal_akhir_data_str, '%Y-%m-%d')
# tambahkan tanggal akhir data dengan jumlah data yang diprediksi
tanggal_akhir_perang = tanggal_akhir_data + datetime.timedelta(days=index)
# print tanggal akhir perang
print('Tanggal akhir perang: ', tanggal_akhir_perang.strftime('%Y-%m-%d'))